In [4]:
%matplotlib inline
import stats_pandas as stats
import pandas as pd
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px

init_notebook_mode(connected=True)

PATH = r'test-chats'

as


In [5]:
import biggest_chats

TOP_N = 10

top_chats = biggest_chats.chat_sizes(PATH)[:TOP_N]
biggest_chats.print_chats(top_chats)

1. Adrian Prorok: 14
2. Adrian Czech: 46


In [6]:
chats_dfs = {chat.name: stats.load_from_path(chat.path) for chat in top_chats}
grouped = {name: stats.groupby_date(chat_df) for name, chat_df in chats_dfs.items()}


# TODO: Plan --> pd.merge (how=outer) na datafrejmach w formacie wide (kolumny --> daty,  wiersze -> Nazwa czatu, w komórkach liczba wiadomości), powinny być nulle tam gdzie sie rozjeżdzą więcorbimy .fillna(0) i jeszcze powinno sie dać zrobić agregacje po wierszach dla wyścigowego charta
# przypal robic chyba N joinow? moze lpeiej byloby zrobic dfy z kolumnami (month, messages, chat) i indeksami domyslnymi i  je po concatowac w łatwy sposob i pozniej na tym wywolac pd.melt(rows=chat, columns=month) i sie jakos samo ogarnie xd 

# dates = stats.groupby_date(chat, 'M')
#
# f = px.line(dates, y = dates, x = dates.index, labels = {'y': 'Message count', 'x': 'Date'})
# f.update_xaxes(nticks = 20, title ='Date')
# f.show()

In [2]:
print('ass')

ass


In [22]:
test_df = grouped['Adrian Czech']

In [50]:
grouped['Adrian Prorok']

timestamp_ms
2022-02-28 00:00:00+01:00    14
Freq: M, Name: timestamp_ms, dtype: int64

In [36]:
np.array(test_df.index)

array([Timestamp('2018-11-30 00:00:00+0100', tz='CET', freq='M'),
       Timestamp('2018-12-31 00:00:00+0100', tz='CET', freq='M'),
       Timestamp('2019-01-31 00:00:00+0100', tz='CET', freq='M'),
       Timestamp('2019-02-28 00:00:00+0100', tz='CET', freq='M'),
       Timestamp('2019-03-31 00:00:00+0100', tz='CET', freq='M')],
      dtype=object)

In [46]:
import numpy as np
# test_df2 = pd.DataFrame(test_df)
# test_df2 = pd.DataFrame(data=[test_df, test_df.index, ['Adi']*5], columns=['messages', 'month', 'name'])
#d = np.array([np.array(test_df.values), np.array(test_df.index), np.array(['Adi']*5)])
d = [test_df.values, test_df.index,['Adrian Czech']*5]
test_df2 = pd.DataFrame(d, index=['messages', 'month', 'name']).T
test_df2

,messages,month,name
0,15,2018-11-30 00:00:00+01:00,Adrian Czech
1,0,2018-12-31 00:00:00+01:00,Adrian Czech
2,0,2019-01-31 00:00:00+01:00,Adrian Czech
3,0,2019-02-28 00:00:00+01:00,Adrian Czech
4,31,2019-03-31 00:00:00+01:00,Adrian Czech


In [77]:
test_df =  grouped['Adrian Czech']
#test_df.cumsum()
d = [test_df.values, test_df.index,['Adrian Prorok']*1]
test_df3 = pd.DataFrame(d, index=['messages', 'month', 'name']).T
test_df3['messages'] = test_df3['messages'].cumsum()
test_df3['month'] = test_df3['month'].dt.strftime('%m/%Y')
test_df3

,messages,month,name
0,15,11/2018,Adrian Prorok
1,15,12/2018,None
2,15,01/2019,None
3,15,02/2019,None
4,46,03/2019,None


In [52]:
pd.concat([test_df2, test_df3])

,messages,month,name
0,15,2018-11-30 00:00:00+01:00,Adrian Czech
1,0,2018-12-31 00:00:00+01:00,Adrian Czech
2,0,2019-01-31 00:00:00+01:00,Adrian Czech
3,0,2019-02-28 00:00:00+01:00,Adrian Czech
4,31,2019-03-31 00:00:00+01:00,Adrian Czech
0,14,2022-02-28 00:00:00+01:00,Adrian Prorok


In [54]:
pd.melt(pd.concat([test_df2, test_df3]), id_vars='name', value_vars='month')

,name,variable,value
0,Adrian Czech,month,2018-11-30 00:00:00+01:00
1,Adrian Czech,month,2018-12-31 00:00:00+01:00
2,Adrian Czech,month,2019-01-31 00:00:00+01:00
3,Adrian Czech,month,2019-02-28 00:00:00+01:00
4,Adrian Czech,month,2019-03-31 00:00:00+01:00
5,Adrian Prorok,month,2022-02-28 00:00:00+01:00


In [56]:
pd.pivot(pd.concat([test_df2, test_df3]), index='name', columns='month', values='messages').fillna(0)

month,2018-11-30 00:00:00+01:00,2018-12-31 00:00:00+01:00,2019-01-31 00:00:00+01:00,2019-02-28 00:00:00+01:00,2019-03-31 00:00:00+01:00,2022-02-28 00:00:00+01:00
name,,,,,,
Adrian Czech,15,0,0,0,31,0
Adrian Prorok,0,0,0,0,0,14


In [83]:
big_df = pd.DataFrame()
for name, msg_by_month in grouped.items():
    data = [msg_by_month.values, msg_by_month.index, [name]*len(msg_by_month)]
    wide_df = pd.DataFrame(data, index=['messages', 'month', 'name']).T
    wide_df['month'] = wide_df['month'].dt.strftime('%Y/%m')
    #wide_df['messages'] = wide_df['messages'].cumsum()
    big_df = pd.concat([big_df, wide_df])

#big_df
result = (pd.pivot(big_df, index='name', columns='month', values='messages')
          .fillna(0)
          .cumsum(axis=1)
          )

result.to_csv('result.csv')
result

month,2018/11,2018/12,2019/01,2019/02,2019/03,2022/02
name,,,,,,
Adrian Czech,15,15,15,15,46,46
Adrian Prorok,0,0,0,0,0,14
